In [118]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [119]:
#import csv data
nba_data = pd.read_csv('nba2023-2024.csv', encoding='latin1', sep=';')

In [120]:
#display data
nba_data.info

<bound method DataFrame.info of       Rk                    Player    Pos  Age   Tm   G  GS    MP    FG   FGA  \
0      1          Precious Achiuwa   PF-C   24  TOT  74  18  21.9   3.2   6.3   
1      1          Precious Achiuwa      C   24  TOR  25   0  17.5   3.1   6.8   
2      1          Precious Achiuwa     PF   24  NYK  49  18  24.2   3.2   6.1   
3      2               Bam Adebayo      C   26  MIA  71  71  34.0   7.5  14.3   
4      3              Ochai Agbaji     SG   23  TOT  78  28  21.0   2.3   5.6   
5      3              Ochai Agbaji     SG   23  UTA  51  10  19.7   2.1   4.9   
6      3              Ochai Agbaji     SG   23  TOR  27  18  23.6   2.7   6.8   
7      4              Santi Aldama     PF   23  MEM  61  35  26.5   4.0   9.3   
8      5  Nickeil Alexander-Walker     SG   25  MIN  82  20  23.4   2.9   6.6   
9      6             Grayson Allen     SG   28  PHO  75  74  33.5   4.5   9.1   
10     7             Jarrett Allen      C   25  CLE  77  77  31.7   6.7  10.6

In [121]:
#display data
nba_data.head

<bound method NDFrame.head of       Rk                    Player    Pos  Age   Tm   G  GS    MP    FG   FGA  \
0      1          Precious Achiuwa   PF-C   24  TOT  74  18  21.9   3.2   6.3   
1      1          Precious Achiuwa      C   24  TOR  25   0  17.5   3.1   6.8   
2      1          Precious Achiuwa     PF   24  NYK  49  18  24.2   3.2   6.1   
3      2               Bam Adebayo      C   26  MIA  71  71  34.0   7.5  14.3   
4      3              Ochai Agbaji     SG   23  TOT  78  28  21.0   2.3   5.6   
5      3              Ochai Agbaji     SG   23  UTA  51  10  19.7   2.1   4.9   
6      3              Ochai Agbaji     SG   23  TOR  27  18  23.6   2.7   6.8   
7      4              Santi Aldama     PF   23  MEM  61  35  26.5   4.0   9.3   
8      5  Nickeil Alexander-Walker     SG   25  MIN  82  20  23.4   2.9   6.6   
9      6             Grayson Allen     SG   28  PHO  75  74  33.5   4.5   9.1   
10     7             Jarrett Allen      C   25  CLE  77  77  31.7   6.7  10.6  

In [122]:
#separating the team with the best win/loss record of the 2023-2024 season
nba_data[nba_data.Tm == 'BOS']

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
35,27,Dalano Banton,PG,24,BOS,24,1,7.1,0.8,2.1,0.373,0.1,0.7,0.125,0.7,1.5,0.486,0.392,0.7,0.8,0.800,0.5,1.0,1.5,0.8,0.2,0.1,0.4,0.8,2.3
92,63,Oshae Brissett,SF,25,BOS,55,1,11.5,1.2,2.8,0.444,0.3,1.0,0.273,1.0,1.8,0.541,0.493,0.9,1.5,0.602,1.1,1.8,2.9,0.8,0.3,0.1,0.4,1.0,3.7
102,71,Jaylen Brown,SF,27,BOS,70,70,33.5,9.0,17.9,0.499,2.1,5.9,0.354,6.9,12.1,0.570,0.557,3.0,4.3,0.703,1.2,4.3,5.5,3.6,1.2,0.5,2.4,2.6,23.0
157,118,JD Davison,SG,21,BOS,8,0,4.9,0.6,1.5,0.417,0.4,0.9,0.429,0.3,0.6,0.400,0.542,0.4,0.5,0.750,0.3,1.0,1.3,1.3,0.1,0.1,0.3,0.5,2.0
278,209,Sam Hauser,SF,26,BOS,79,13,22.0,3.2,7.1,0.446,2.5,5.9,0.424,0.7,1.2,0.559,0.623,0.2,0.2,0.895,0.6,2.9,3.5,1.0,0.5,0.3,0.4,1.3,9.0
295,222,Jrue Holiday,PG,33,BOS,69,69,32.8,4.8,10.0,0.480,2.0,4.7,0.429,2.8,5.3,0.526,0.581,0.9,1.0,0.833,1.2,4.2,5.4,4.8,0.9,0.8,1.8,1.6,12.5
302,227,Al Horford,C,37,BOS,65,33,26.8,3.3,6.4,0.511,1.7,4.0,0.419,1.6,2.5,0.658,0.640,0.4,0.5,0.867,1.3,5.1,6.4,2.6,0.6,1.0,0.7,1.4,8.6
360,283,Luke Kornet,C,28,BOS,63,7,15.6,2.3,3.2,0.700,0.0,0.0,1.000,2.2,3.2,0.698,0.702,0.8,0.9,0.907,1.9,2.3,4.1,1.1,0.4,1.0,0.3,1.2,5.3
479,368,Svi Mykhailiuk,SF,26,BOS,41,2,10.1,1.4,3.3,0.416,1.0,2.6,0.389,0.4,0.7,0.517,0.569,0.1,0.2,0.667,0.3,1.0,1.2,0.9,0.3,0.0,0.3,0.5,4.0
515,394,Drew Peterson,PF,24,BOS,3,0,7.7,1.3,2.0,0.667,1.0,1.7,0.600,0.3,0.3,1.000,0.917,0.0,0.0,0.000,0.0,0.3,0.3,0.3,0.7,0.0,0.3,0.3,3.7


In [123]:
#looking for any missing values
missing = pd.concat([nba_data.isnull().sum(), 100 * nba_data.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count', ascending = False)

,count,%
Rk,0,0.0
Player,0,0.0
PF,0,0.0
TOV,0,0.0
BLK,0,0.0
STL,0,0.0
AST,0,0.0
TRB,0,0.0
DRB,0,0.0
ORB,0,0.0


In [124]:
#looking for duplicates
nba_data['Player'].value_counts()

Player
Mike Muscala                4
Dylan Windler               4
Kenneth Lofton Jr.          4
Kira Lewis Jr.              4
Danilo Gallinari            4
Malachi Flynn               4
Shake Milton                4
Dennis Schröder             3
Tosan Evbuomwan             3
Simone Fontecchio           3
Precious Achiuwa            3
Evan Fournier               3
Pascal Siakam               3
Terry Rozier                3
Daniel Gafford              3
Ryan Rollins                3
David Roddy                 3
Alex Fudge                  3
Mamadi Diakite              3
Spencer Dinwiddie           3
Harry Giles                 3
Seth Curry                  3
Jaden Springer              3
Lamar Stevens               3
Robert Covington            3
Daniel Theis                3
Xavier Tillman Sr.          3
P.J. Tucker                 3
Alec Burks                  3
Troy Brown Jr.              3
Taj Gibson                  3
Immanuel Quickley           3
Jacob Gilyard               3
Jor

In [125]:
#I noticed that there were multiple entries to players... why? 
#The Tm(team) column is different, this indicates the player played on different teams during the season; however there is also another row where the team name appears as TOT(total) which is the sum of the stats of the players that played in different teams during the season.
#In order to organize the data and only keep players entries unique, I will drop the separate rows that don't contain the TOT accronym, and then replace the TOT accronym with the accronyms of the teams they played in during this season.

#Create a map from players to teams (excluding 'TOT' entries)
player_teams = nba_data[nba_data['Tm'] != 'TOT'].groupby('Player')['Tm'].unique().apply(lambda x: ','.join(x))

#Map the player_teams to the 'Tm' column where 'Tm' is 'TOT'
nba_data.loc[nba_data['Tm'] == 'TOT', 'Tm'] = nba_data['Player'].map(player_teams)

In [126]:
print(nba_data.info)

<bound method DataFrame.info of       Rk                    Player    Pos  Age           Tm   G  GS    MP  \
0      1          Precious Achiuwa   PF-C   24      TOR,NYK  74  18  21.9   
1      1          Precious Achiuwa      C   24          TOR  25   0  17.5   
2      1          Precious Achiuwa     PF   24          NYK  49  18  24.2   
3      2               Bam Adebayo      C   26          MIA  71  71  34.0   
4      3              Ochai Agbaji     SG   23      UTA,TOR  78  28  21.0   
5      3              Ochai Agbaji     SG   23          UTA  51  10  19.7   
6      3              Ochai Agbaji     SG   23          TOR  27  18  23.6   
7      4              Santi Aldama     PF   23          MEM  61  35  26.5   
8      5  Nickeil Alexander-Walker     SG   25          MIN  82  20  23.4   
9      6             Grayson Allen     SG   28          PHO  75  74  33.5   
10     7             Jarrett Allen      C   25          CLE  77  77  31.7   
11     8               Timmy Allen     SF   

In [127]:
#Now I need to remove multiple entries of players, and keep only the first ones which show all teams were they played, and their total scores for the entire season. 
player_counts = nba_data['Player'].value_counts()

#Players appearing more than once have multiple team entries
players_multiple_entries = player_counts[player_counts > 1].index

#Filter to keep rows where 'Tm' contains a comma or the player does not have multiple entries
nba_data_filtered = nba_data[(nba_data['Player'].isin(players_multiple_entries) & nba_data['Tm'].str.contains(',')) | ~nba_data['Player'].isin(players_multiple_entries)]

#Drop duplicate players
nba_data_final = nba_data_filtered.drop_duplicates(subset='Player', keep='first')

In [128]:
print(nba_data_final.info)

<bound method DataFrame.info of       Rk                    Player    Pos  Age           Tm   G  GS    MP  \
0      1          Precious Achiuwa   PF-C   24      TOR,NYK  74  18  21.9   
3      2               Bam Adebayo      C   26          MIA  71  71  34.0   
4      3              Ochai Agbaji     SG   23      UTA,TOR  78  28  21.0   
7      4              Santi Aldama     PF   23          MEM  61  35  26.5   
8      5  Nickeil Alexander-Walker     SG   25          MIN  82  20  23.4   
9      6             Grayson Allen     SG   28          PHO  75  74  33.5   
10     7             Jarrett Allen      C   25          CLE  77  77  31.7   
11     8               Timmy Allen     SF   24          MEM   5   0  25.0   
12     9             Jose Alvarado     PG   25          NOP  56   0  18.4   
13    10             Kyle Anderson     PF   30          MIN  79  10  22.6   
14    11     Giannis Antetokounmpo     PF   29          MIL  73  73  35.2   
15    12    Thanasis Antetokounmpo     PF   

In [129]:
nba_data_final = nba_data_final.drop(columns='Rk')

In [130]:
print(nba_data_final.columns)

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')


In [ ]:
#creating new columns